# Decoradores de clase factory

Vamos a importar las siguientes clases y decoradores para trabajar en este notebook:

In [1]:
import inspect

In [2]:
def typename(obj):
    return type(obj).__name__

### Decorador auto_repr

In [3]:
def auto_repr(cls):
    
    miembros = vars(cls)
    
    for nombre, miembro in miembros.items():
        print(nombre, miembro)
    
    if "__repr__" in miembros:
        raise TypeError(f"La clase {cls.__name__} ya define el propio __repr__")
    
    if "__init__" not in miembros:
        raise TypeError(f"La clase {cls.__name__} debe tener un método __init__")
        
    sig = inspect.signature(cls.__init__)
    
    parametros = list(sig.parameters)[1:]
    
    # Lo que hacemos aquí es decirle: 
    # 1. Para cada property, intenta obtenerla del diccionario miembros
    # 2. Si no está, devuelve None
    # Si para alguno de los valores devuelve None, es que ese valor no tiene un property establecido
    
    if not all(isinstance(miembros.get(nombre, None), property) for nombre in parametros):
        raise TypeError("Todos los argumentos de __init__ deben tener una property establecida.")
    
    def repr_sintetizado(self):
    
        return "{typename}({args})".format(
            typename=typename(self),
            args=", ".join(
                "{nombre}={valor}".format(
                    nombre=nombre,
                    valor=getattr(self, nombre)
                ) for nombre in parametros
            )
        )

    setattr(cls, "__repr__", repr_sintetizado)
    
    return cls

### Clase Localizacion

In [4]:
@auto_repr
class Localizacion:
    
    def __init__(self, nombre, posicion):
        
        self._nombre = nombre
        self._posicion = posicion
        
    @property
    def nombre(self):
        return self._nombre
    
    @property
    def posicion(self):
        return self._posicion
    
    def __str__(self):
        return self.nombre

__module__ __main__
__init__ <function Localizacion.__init__ at 0x0000023D9A147310>
nombre <property object at 0x0000023D9A06EB30>
posicion <property object at 0x0000023D9A14D6D0>
__str__ <function Localizacion.__str__ at 0x0000023D9A1474C0>
__dict__ <attribute '__dict__' of 'Localizacion' objects>
__weakref__ <attribute '__weakref__' of 'Localizacion' objects>
__doc__ None


In [5]:
hong_kong = Localizacion("Hong Kong", (22.29, 114.16))
estocolmo = Localizacion("Estocolmo", (59.33, 18.06))
ciudad_del_cabo = Localizacion("Cape Town", (-33.93, 18.42))
rotterdam = Localizacion("Rotterdam", (51.96, 4.47))
maracaibo = Localizacion("Maracaibo", (10.65, -71.65))

Ahora vamos a crear una clase llamada **Itinerario**, que cree un itinerario de viaje a partir de una serie de localizaciones:

In [6]:
@auto_repr
class Itinerario:

    @classmethod
    def desde_localizaciones(cls, *localizaciones):
        return cls(localizaciones)

    def __init__(self, localizaciones):
        self._localizaciones = list(localizaciones)

    def __str__(self):
        return "\n".join(localizacion.nombre for localizacion in self._localizaciones)

    @property
    def localizaciones(self):
        return tuple(self._localizaciones)

    @property
    def origen(self):
        return self._localizaciones[0]

    @property
    def destino(self):
        return self._localizaciones[-1]

    def añadir(self, localizacion):
        self._localizaciones.append(localizacion)

    def borrar(self, nombre):
        borrar_indices = [
            indice for indice, localizacion in enumerate(self._localizaciones)
            if localizacion.nombre == nombre
        ]
        for indice in reversed(borrar_indices):
            del self._localizaciones[indice]

    def truncar_en(self, nombre):
        parar = None
        for indice, localizacion in enumerate(self._localizaciones):
            if localizacion.nombre == nombre:
                parar = indice + 1

        self._localizaciones = self._localizaciones[:parar]

__module__ __main__
desde_localizaciones <classmethod object at 0x0000023D9A146310>
__init__ <function Itinerario.__init__ at 0x0000023D9A147670>
__str__ <function Itinerario.__str__ at 0x0000023D9A1475E0>
localizaciones <property object at 0x0000023D9A13AEF0>
origen <property object at 0x0000023D9A155720>
destino <property object at 0x0000023D9A1557C0>
añadir <function Itinerario.añadir at 0x0000023D9A1471F0>
borrar <function Itinerario.borrar at 0x0000023D9A14E700>
truncar_en <function Itinerario.truncar_en at 0x0000023D9A14E670>
__dict__ <attribute '__dict__' of 'Itinerario' objects>
__weakref__ <attribute '__weakref__' of 'Itinerario' objects>
__doc__ None


Vamos a crear un itinerario de viaje:

In [7]:
viaje = Itinerario.desde_localizaciones(maracaibo, rotterdam, estocolmo)

In [8]:
viaje

Itinerario(localizaciones=(Localizacion(nombre=Maracaibo, posicion=(10.65, -71.65)), Localizacion(nombre=Rotterdam, posicion=(51.96, 4.47)), Localizacion(nombre=Estocolmo, posicion=(59.33, 18.06))))

In [9]:
print(viaje)

Maracaibo
Rotterdam
Estocolmo


In [10]:
viaje.origen

Localizacion(nombre=Maracaibo, posicion=(10.65, -71.65))

In [11]:
viaje.destino

Localizacion(nombre=Estocolmo, posicion=(59.33, 18.06))

In [12]:
viaje.añadir(ciudad_del_cabo)

In [13]:
viaje.añadir(hong_kong)

In [14]:
viaje.borrar("Estocolmo")

In [15]:
print(viaje)

Maracaibo
Rotterdam
Cape Town
Hong Kong


In [16]:
viaje.truncar_en("Rotterdam")

In [17]:
print(viaje)

Maracaibo
Rotterdam


## Decorador *postcondicion*

Una vez hemos probado que la clase itinerario funciona correctamente, vamos a definir el siguiente decorador:

In [18]:
import functools

In [19]:
def postcondicion(predicado):

    def decorador(f):

        @functools.wraps(f)
        def envoltorio(self, *args, **kwargs):
            resultado = f(self, *args, **kwargs)
            if not predicado(self):
                raise RuntimeError(
                    f"Post-condicion {predicado.__name__} no "
                    f"mantenida para {self!r}"
                )
            return resultado

        return envoltorio

    return decorador

Gracias a este decorador, podemos establecer post-condiciones que se deben cumplir en nuestro código:

In [20]:
def al_menos_dos_localizaciones(itinerario):
    return len(itinerario._localizaciones) >= 2

Si ahora añadimos esta post condición a todos los métodos que modifiquen nuestro itinerario **(añadir, borrar, truncar) y al inicializador**, tendremos lo siguiente:

In [21]:
@auto_repr
class Itinerario:

    @classmethod
    def desde_localizaciones(cls, *localizaciones):
        return cls(localizaciones)

    @postcondicion(al_menos_dos_localizaciones)
    def __init__(self, localizaciones):
        self._localizaciones = list(localizaciones)

    def __str__(self):
        return "\n".join(localizacion.nombre for localizacion in self._localizaciones)

    @property
    def localizaciones(self):
        return tuple(self._localizaciones)

    @property
    def origen(self):
        return self._localizaciones[0]

    @property
    def destino(self):
        return self._localizaciones[-1]

    @postcondicion(al_menos_dos_localizaciones)
    def añadir(self, localizacion):
        self._localizaciones.append(localizacion)
        
    @postcondicion(al_menos_dos_localizaciones)
    def borrar(self, nombre):
        borrar_indices = [
            indice for indice, localizacion in enumerate(self._localizaciones)
            if localizacion.nombre == nombre
        ]
        for indice in reversed(borrar_indices):
            del self._localizaciones[indice]

    @postcondicion(al_menos_dos_localizaciones)
    def truncar_en(self, nombre):
        parar = None
        for indice, localizacion in enumerate(self._localizaciones):
            if localizacion.nombre == nombre:
                parar = indice + 1

        self._localizaciones = self._localizaciones[:parar]

__module__ __main__
desde_localizaciones <classmethod object at 0x0000023D9A175790>
__init__ <function Itinerario.__init__ at 0x0000023D9A14F550>
__str__ <function Itinerario.__str__ at 0x0000023D9A14F430>
localizaciones <property object at 0x0000023D9A170360>
origen <property object at 0x0000023D9A17CB30>
destino <property object at 0x0000023D9A17CB80>
añadir <function Itinerario.añadir at 0x0000023D9A14F8B0>
borrar <function Itinerario.borrar at 0x0000023D9A14F9D0>
truncar_en <function Itinerario.truncar_en at 0x0000023D9A14FAF0>
__dict__ <attribute '__dict__' of 'Itinerario' objects>
__weakref__ <attribute '__weakref__' of 'Itinerario' objects>
__doc__ None


Si intentamos crear un itinerario con menos de 2 localizaciones:

In [22]:
try:
    
    viaje = Itinerario.desde_localizaciones(maracaibo)
    
except RuntimeError as error:
    
    print(error)

Post-condicion al_menos_dos_localizaciones no mantenida para Itinerario(localizaciones=(Localizacion(nombre=Maracaibo, posicion=(10.65, -71.65)),))


In [23]:
viaje = Itinerario.desde_localizaciones(maracaibo, rotterdam, ciudad_del_cabo, estocolmo)

In [24]:
print(viaje)

Maracaibo
Rotterdam
Cape Town
Estocolmo


Si ahora intenamos borrar tres de las cuatro localizaciones truncando el itinerario en Maracaibo:

In [25]:
try:
    
    viaje.truncar_en("Maracaibo")

except RuntimeError as error:
    
    print(error)

Post-condicion al_menos_dos_localizaciones no mantenida para Itinerario(localizaciones=(Localizacion(nombre=Maracaibo, posicion=(10.65, -71.65)),))


## Decorador de clase factory *invariante*

Lo que vamos a hacer ahora es crear un decorador de clase factory, es decir, un decorador de clase que aplique decoradores a los métodos de una clase.

Cuando se aplique este decorador a una clase, todos sus métodos tendrán aplicados el decorador **postcondicion** con el predicado que se pase como argumento:

In [26]:
def invariante(predicado):
    
    decorador = postcondicion(predicado)
    
    def decorador_de_clase(cls):
        
        print(cls.__name__)
        
        miembros = list(vars(cls).items())
                
        for nombre, miembro in miembros:
            if inspect.isfunction(miembro):
                miembro_decorado = decorador(miembro)
                setattr(cls, nombre, miembro_decorado)
        
        return cls
    
    return decorador_de_clase

Vamos a eliminar el decorador **postcondicio** de los métodos de la clase y a añadir el decorador **invariante** a la definición de la clase:

In [27]:
@auto_repr
@invariante(al_menos_dos_localizaciones)
class Itinerario:

    @classmethod
    def desde_localizaciones(cls, *localizaciones):
        return cls(localizaciones)

    def __init__(self, localizaciones):
        self._localizaciones = list(localizaciones)

    def __str__(self):
        return "\n".join(localizacion.nombre for localizacion in self._localizaciones)

    @property
    def localizaciones(self):
        return tuple(self._localizaciones)

    @property
    def origen(self):
        return self._localizaciones[0]

    @property
    def destino(self):
        return self._localizaciones[-1]

    def añadir(self, localizacion):
        self._localizaciones.append(localizacion)
        
    def borrar(self, nombre):
        borrar_indices = [
            indice for indice, localizacion in enumerate(self._localizaciones)
            if localizacion.nombre == nombre
        ]
        for indice in reversed(borrar_indices):
            del self._localizaciones[indice]

    def truncar_en(self, nombre):
        parar = None
        for indice, localizacion in enumerate(self._localizaciones):
            if localizacion.nombre == nombre:
                parar = indice + 1

        self._localizaciones = self._localizaciones[:parar]

Itinerario
__module__ __main__
desde_localizaciones <classmethod object at 0x0000023D9A173940>
__init__ <function Itinerario.__init__ at 0x0000023D9A17DD30>
__str__ <function Itinerario.__str__ at 0x0000023D9A17DC10>
localizaciones <property object at 0x0000023D9A1774A0>
origen <property object at 0x0000023D9A189950>
destino <property object at 0x0000023D9A1899A0>
añadir <function Itinerario.añadir at 0x0000023D9A1860D0>
borrar <function Itinerario.borrar at 0x0000023D9A186670>
truncar_en <function Itinerario.truncar_en at 0x0000023D9A186700>
__dict__ <attribute '__dict__' of 'Itinerario' objects>
__weakref__ <attribute '__weakref__' of 'Itinerario' objects>
__doc__ None


Si ahora intentamos crear un itinerario con una sola localización:

In [28]:
try:
    
    viaje = Itinerario.desde_localizaciones(rotterdam, estocolmo)
    
except RuntimeError as error:
    
    print(error)

Podemos ver como se ha aplicado al inicializador.

Si ahora creamos un itinerario con mas de dos localizaciones e intentamos quitar una:

In [29]:
viaje = Itinerario.desde_localizaciones(rotterdam, estocolmo)

try:
    
    viaje = Itinerario.desde_localizaciones(rotterdam, estocolmo)
    viaje.borrar("Estocolmo")
    
except RuntimeError as error:
    
    print(error)

Post-condicion al_menos_dos_localizaciones no mantenida para Itinerario(localizaciones=(Localizacion(nombre=Rotterdam, posicion=(51.96, 4.47)),))


Vemos como salta el error, lo que quiere decir que se ha aplicado al método borrar también.

## Decorador *no_duplicados*:

Vamos a crear un decorador que no permita tener localizaciones duplicadas en el itinerario y vamos a aplicarlo:

In [30]:
def no_duplicados(itinerario):
    return False if len(itinerario.localizaciones) > len(set(itinerario.localizaciones)) else True

In [31]:
@auto_repr
@invariante(no_duplicados)
@invariante(al_menos_dos_localizaciones)
class Itinerario:

    @classmethod
    def desde_localizaciones(cls, *localizaciones):
        return cls(localizaciones)

    def __init__(self, localizaciones):
        self._localizaciones = list(localizaciones)

    def __str__(self):
        return "\n".join(localizacion.nombre for localizacion in self._localizaciones)

    @property
    def localizaciones(self):
        return tuple(self._localizaciones)

    @property
    def origen(self):
        return self._localizaciones[0]

    @property
    def destino(self):
        return self._localizaciones[-1]

    def añadir(self, localizacion):
        self._localizaciones.append(localizacion)
        
    def borrar(self, nombre):
        borrar_indices = [
            indice for indice, localizacion in enumerate(self._localizaciones)
            if localizacion.nombre == nombre
        ]
        for indice in reversed(borrar_indices):
            del self._localizaciones[indice]

    def truncar_en(self, nombre):
        parar = None
        for indice, localizacion in enumerate(self._localizaciones):
            if localizacion.nombre == nombre:
                parar = indice + 1

        self._localizaciones = self._localizaciones[:parar]

Itinerario
Itinerario
__module__ __main__
desde_localizaciones <classmethod object at 0x0000023D9A113DC0>
__init__ <function Itinerario.__init__ at 0x0000023D9A157280>
__str__ <function Itinerario.__str__ at 0x0000023D9A1573A0>
localizaciones <property object at 0x0000023D9A0B6F90>
origen <property object at 0x0000023D9A18A090>
destino <property object at 0x0000023D9A18A130>
añadir <function Itinerario.añadir at 0x0000023D9A157940>
borrar <function Itinerario.borrar at 0x0000023D9A157DC0>
truncar_en <function Itinerario.truncar_en at 0x0000023D9A157E50>
__dict__ <attribute '__dict__' of 'Itinerario' objects>
__weakref__ <attribute '__weakref__' of 'Itinerario' objects>
__doc__ None


In [32]:
try:
    viaje = Itinerario.desde_localizaciones(estocolmo, maracaibo, estocolmo)

except RuntimeError as error:
    print(error)

Post-condicion no_duplicados no mantenida para Itinerario(localizaciones=(Localizacion(nombre=Estocolmo, posicion=(59.33, 18.06)), Localizacion(nombre=Maracaibo, posicion=(10.65, -71.65)), Localizacion(nombre=Estocolmo, posicion=(59.33, 18.06))))
